In [ ]:
#!/usr/bin/env conda run -n datimacchinaenv python
import sys
print(sys.executable)    ### C:\ProgramData\Anaconda3_1\envs\datimacchinaenv\python.exe

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))
!jupyter kernelspec list

In [ ]:
# !{sys.executable} -m pip install xlwt
# !{sys.executable} -m pip install openpyxl
# !{sys.executable} -m pip install Pillow
# !{sys.executable} -m pip install seaborn
# !conda install --yes --prefix {sys.prefix} xlrd 
# !{sys.executable} -m pip install tables     
# !{sys.executable} -m pip install simpledbf
import sys
import pandas as pd
pd.options.display.max_columns = None  ## in order to show all columns
import os
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np
from pylab import *
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import IPython.display as IPdisplay
import shutil
# per matrice correlazione
import seaborn as sns
# Scipy helper functions
from scipy.stats import percentileofscore
from scipy import stats
import matplotlib.tri as mtri
import datetime as dt
import h5py
import warnings
warnings.filterwarnings('ignore')
from matplotlib.pyplot import figure
import xlwt  # necessario per esportare xls con pandas
from scipy.interpolate import griddata
from datetime import datetime, timedelta

In [ ]:
import glob
from PIL import Image
from images2gif import writeGif  # requires --> (datimacchinaenv) pip install images2gif

#####   important before to start do the same change to the package images2gif   ###########

#       manually change the two import statements in your already installed package (in __init__.py):

#       -from images2gif import readGif as readGif
#       -from images2gif import writeGif as writeGif
#       +from .images2gif import readGif as readGif
#       +from .images2gif import writeGif as writeGif

### LOAD FILES

In [ ]:
path = os.getcwd()+'\\DATA\\20200107'

files = []
filespath = []
# r=root, d=directories, f = files
for r, d, f in os.walk(path):
    for file in f:
        if r'MERGE' in file:
            files.append(file)
            filespath.append(os.path.join(r, file))
            
# since there is only one file I need only the file name
os.path.join(r, files[0])

print(files)
print(filespath)
df = pd.read_excel(os.path.join(r, files[0]))
print('file loaded'+files[0])
print('\nColumns list:')
print(df.columns.tolist())

### LOAD MULTI FILE

In [ ]:
# path = os.getcwd()+'\\DATA\\1048'
path = os.getcwd()+'\\DATA2'

files = []
filespath = []
# r=root, d=directories, f = files
for r, d, f in os.walk(path):
    for file in f:
        if '.xls' in file:
            files.append(file)
            filespath.append(os.path.join(r, file))
            
# since there is only one file I need only the file name
os.path.join(r, files[0])
#check the first and last 5 files if correctly in list "files"
print('primi 5 files',files[:5])
print('ultimi 6 files',files[-6:])
print('primi 2 filespath',filespath[:2])

In [ ]:
dfxls=pd.DataFrame({})
n=0
for file in filespath:
    try:
        xlstemp = pd.read_excel(file, 'Sheet0')
        #xlstemp['CPTname']=file.replace('.xls','')
        dfxls = dfxls.append(xlstemp, ignore_index = True, sort=True)
        n+=1
        if n%10==0: print(n)
    except:
        print('A file was not loaded')
%reset_selective -f "^xlstemp$"

In [ ]:
dfxls

In [ ]:
dfxls.describe()

###  SAVE AND LOAD AS H5

In [ ]:
dfxls.to_hdf('1048_2020.h5', key='losses')

In [ ]:
dfxls = pd.read_hdf('1048_2020.h5', key='losses')

In [ ]:
df = dfxls.copy()

### CLEAN DATA AND CREATE NEW COLUMN

In [ ]:
%%time
#### Drop double columns ending by .1

df.filter(regex='[.]1$')
df = df[df.columns.drop(list(df.filter(regex='[.]1$')))]  # per eliminare eventuali colonne doppie

###  Create new columns

#### First we create a column that can be detected as date

df['datestamp']=pd.to_datetime(df.Date)
df['timestamp']=df.datestamp.apply(lambda x: int(time.mktime(x.timetuple())))

df.sort_values('timestamp',inplace=True)

#### Then we find the position of relevant indicators  --> do not remove spaces before this 

# cerco la posizione delle colonne principali
 
# regex  --> (?i) means case unsensible

pos_filllev=int(df.columns.get_loc(df.filter(regex='(?i)fill.*lev.*work.*chamb').columns.tolist()[0]))
pos_stroke=int(df.columns.get_loc(df.filter(regex='(?i)aet.*stroke').columns.tolist()[0]))
pos_p01=int(df.columns.get_loc(df.filter(regex='^(?=.*(?i)P0.1)(?=.*(?i)Flow)(?=.*(?i)m³/h).*$').columns.tolist()[0]))
pos_p02=int(df.columns.get_loc(df.filter(regex='^(?=.*(?i)P0.2)(?=.*(?i)Flow)(?=.*(?i)m³/h).*$').columns.tolist()[0]))
pos_p03=int(df.columns.get_loc(df.filter(regex='^(?=.*(?i)P0.3)(?=.*(?i)Flow)(?=.*(?i)m³/h).*$').columns.tolist()[0]))
pos_p04=int(df.columns.get_loc(df.filter(regex='^(?=.*(?i)P0.4)(?=.*(?i)Flow)(?=.*(?i)m³/h).*$').columns.tolist()[0]))
pos_p05=int(df.columns.get_loc(df.filter(regex='^(?=.*(?i)P0.5)(?=.*(?i)Flow)(?=.*(?i)m³/h).*$').columns.tolist()[0]))
# non presenti nella lista erano usate per correggere il feed 
# pos_pos02=int(df.columns.get_loc(df.filter(regex='^(?=.*(?i)posit)(?=.*(?i)2)(?=.*(?i)m³/h).*$').columns.tolist()[0]))
# pos_pos03=int(df.columns.get_loc(df.filter(regex='^(?=.*(?i)posit)(?=.*(?i)3)(?=.*(?i)m³/h).*$').columns.tolist()[0]))
pos_advspeed=int(df.columns.get_loc(df.filter(regex='(?i)Advance.*speed.*').columns.tolist()[0]))
pos_torque=int(df.columns.get_loc(df.filter(regex='(?i)torque.*cutting.*wheel.*').columns.tolist()[0]))
pos_feed=int(df.columns.get_loc(df.filter(regex='^(?=.*(?i)flow)(?=.*(?i)feed)(?=.*(?i)m³/h).*$').columns.tolist()[0]))
pos_slurry=int(df.columns.get_loc(df.filter(regex='^(?=.*(?i)slurry)(?=.*(?i)line)(?=.*(?i)m³/h).*$').columns.tolist()[0]))
pos_num=int(df.columns.get_loc(df.filter(regex='^(?=.*(?i)1048)(?=.*(?i)advance)(?=.*(?i)[-]).*$').columns.tolist()[0]))
pos_prch01=int(df.columns.get_loc(df.filter(regex='^(?=.*(?i)chamb)(?=.*(?i)1)(?=.*(?i)bar).*$').columns.tolist()[0]))
pos_prch02=int(df.columns.get_loc(df.filter(regex='^(?=.*(?i)chamb)(?=.*(?i)2)(?=.*(?i)bar).*$').columns.tolist()[0]))
pos_prch03=int(df.columns.get_loc(df.filter(regex='^(?=.*(?i)chamb)(?=.*(?i)3)(?=.*(?i)bar).*$').columns.tolist()[0]))
pos_prch04=int(df.columns.get_loc(df.filter(regex='^(?=.*(?i)chamb)(?=.*(?i)4)(?=.*(?i)bar).*$').columns.tolist()[0]))
pos_prch05=int(df.columns.get_loc(df.filter(regex='^(?=.*(?i)chamb)(?=.*(?i)5)(?=.*(?i)bar).*$').columns.tolist()[0]))
pos_prch06=int(df.columns.get_loc(df.filter(regex='^(?=.*(?i)chamb)(?=.*(?i)6)(?=.*(?i)bar).*$').columns.tolist()[0]))
pos_prch07=int(df.columns.get_loc(df.filter(regex='^(?=.*(?i)chamb)(?=.*(?i)7)(?=.*(?i)bar).*$').columns.tolist()[0]))
pos_prch08=int(df.columns.get_loc(df.filter(regex='^(?=.*(?i)chamb)(?=.*(?i)8)(?=.*(?i)bar).*$').columns.tolist()[0]))
pos_date=int(df.columns.get_loc(df.filter(regex='datestamp').columns.tolist()[0]))
pos_timestamp=int(df.columns.get_loc(df.filter(regex='timestamp').columns.tolist()[0]))
# densities
pos_dens_p04=int(df.columns.get_loc(df.filter(regex='^(?=.*(?i)dens)(?=.*(?i)p04)(?=.*(?i)t/m).*$').columns.tolist()[0]))
pos_dens_slurry=int(df.columns.get_loc(df.filter(regex='^(?=.*(?i)dens)(?=.*(?i)slurr)(?=.*(?i)line)(?=.*(?i)t/m³).*$').columns.tolist()[0]))
pos_dens_feed=int(df.columns.get_loc(df.filter(regex='^(?=.*(?i)dens)(?=.*(?i)feed)(?=.*(?i)line)(?=.*(?i)t/m³).*$').columns.tolist()[0]))
# stroke cyl abcde
pos_cy_a=int(df.columns.get_loc(df.filter(regex='^(?=.*(?i)strok)(?=.*(?i)cyl)(?=.*(?i) A ).*$').columns.tolist()[0]))
pos_cy_b=int(df.columns.get_loc(df.filter(regex='^(?=.*(?i)strok)(?=.*(?i)cyl)(?=.*(?i) B ).*$').columns.tolist()[0]))
pos_cy_c=int(df.columns.get_loc(df.filter(regex='^(?=.*(?i)strok)(?=.*(?i)cyl)(?=.*(?i) C ).*$').columns.tolist()[0]))
pos_cy_d=int(df.columns.get_loc(df.filter(regex='^(?=.*(?i)strok)(?=.*(?i)cyl)(?=.*(?i) D ).*$').columns.tolist()[0]))
pos_cy_e=int(df.columns.get_loc(df.filter(regex='^(?=.*(?i)strok)(?=.*(?i)cyl)(?=.*(?i) E ).*$').columns.tolist()[0]))

col_list=df.columns.tolist()
cl = col_list.copy()  # per chiamare la lista in maniera veloce



## Creo colonne

- **'datestamp_diff'** delta t tra righe in ore  **dt_h** nomesemplificato
- **'stroke_diff[mm]'**  delta stroke corretto AET
- **'stroke_diff_a[mm]'**  delta stroke a,b,c,d,e per ogni pistone

- **'state1'** 1 se tutti i pistoni avanzano
- **'state2'** 1 se AET net stroke avanza

- **'state'** output 'fulloper','stall_rotating','stall_stand'

- **'dfxls['vol_teor_cum[m³]']'**
- **'dfxls['vol_teor_cum_dry[m³]']'**
- **'dfxls['dvol_teor[m³]']'**

- **'dfxls['dvol_calc[m³]']'**
- **'dfxls['dvol_calc_MthE[m³]']'**

- **'dfxls['dvol_teor-calc[m³]']'**   dvol_terico - dvol_calc


- **col_list**  is a list for fast detection of columns
- **pos_dict**  is a dictionary for fast detection of new created columns



In [ ]:
# reinitialize the dataframe
dfxls = df[(df.Date>"2020-01-03")&(df.Date<"2020-01-04")].copy()#     <-- Attenzione 
del df

In [ ]:
%%time
# Dati di input
nw = 0.25 # indice dei vuoti
rhosoil_ = 2.10 # t/m3 terreno in sito
rhow = 1
rhobent = 2.2
Area_Cutter = 39.59

## We create new columns

#### Datestamp diff (aa-mm-gg hh-mm-ss)
#### Datestamp diff [h]

dfxls['datestamp_p'] = dfxls.datestamp.shift(1).copy()
# Beign the first row == 0 we need to delete it _ but we will do it at the end otherwise we lose data
#dfxls=dfxls.dropna().copy()

dfxls['datestamp_diff']= dfxls['datestamp']-dfxls['datestamp_p']
# dfxls['datestamp_diff2'] = dfxls['datestamp'].diff(periods=1).copy()  # stesso risultato come sopra

dfxls['datestamp_diff[h]'] = dfxls['datestamp_diff'].astype('timedelta64[s]').fillna(0).copy()/3600
dfxls['dt_h'] = dfxls['datestamp_diff[h]'].copy()

#### Differenziali dpressure chambers loop sulle 8 celle di pressione

for i in np.arange(1,9,1):
    dfxls['dpress_ch{}'.format(i)] = dfxls[col_list[pos_prch01+i-1]].diff(periods=1).copy()

#### Creo colonna stroke - delta stroke solo positivi - volumi teorici e volumi teorici cumulati        ['dVol_Teor[m³]']

dfxls['stroke_diff[mm]'] = dfxls[col_list[pos_stroke]].diff(periods=1).fillna(0).copy()
dfxls['stroke_diff_a[mm]'] = dfxls[col_list[pos_cy_a]].diff(periods=1).fillna(0).copy()
dfxls['stroke_diff_b[mm]'] = dfxls[col_list[pos_cy_a]].diff(periods=1).fillna(0).copy()
dfxls['stroke_diff_c[mm]'] = dfxls[col_list[pos_cy_a]].diff(periods=1).fillna(0).copy()
dfxls['stroke_diff_d[mm]'] = dfxls[col_list[pos_cy_a]].diff(periods=1).fillna(0).copy()
dfxls['stroke_diff_e[mm]'] = dfxls[col_list[pos_cy_a]].diff(periods=1).fillna(0).copy()


# creazione indici di movimento
def ff(row):
    
    if row['stroke_diff[mm]']>0 and row['stroke_diff_a[mm]']>0 and row['stroke_diff_b[mm]']>0 and\
       row['stroke_diff_c[mm]']>0 and row['stroke_diff_d[mm]']>0\
       and row['stroke_diff_e[mm]']>0:
        return 1
    else:
        return 0

dfxls['state1'] = dfxls.apply(ff, axis=1)     # 0 se stop 1 se tutti i jack sono in movimento
dfxls['dstroke[mm]'] = dfxls['AET Net Stroke [mm]'].diff(periods=1).fillna(0)  ## identico a dfxls['stroke_diff[mm]']
dfxls['state2'] = dfxls['dstroke[mm]'].map(lambda x: 1 if x>0 else 0) # 1 se dstroke avanza 0 se fermo o negativo



#####  Volumi dallo stroke
    
dfxls['dstroke_pos[mm]'] =dfxls[col_list[pos_stroke]].diff(periods=1).fillna(0).map(lambda x: 0 if x<0 else x).copy()   #è il dstroke solo positivo


dfxls['nw'] = nw

dfxls['vol_teor_cum[m³]'] = dfxls.iloc[:,pos_stroke]/1000*Area_Cutter    # use only to calculate dvol
dfxls['vol_teor_cum_dry[m³]'] = dfxls['vol_teor_cum[m³]']*(1-dfxls.nw)         # use only to calculate dvol

dfxls['dvol_teor[m³]'] = dfxls['vol_teor_cum[m³]'].diff(periods=1).fillna(0).map(lambda x: 0 if x<0 else x).copy()
dfxls['dvol_teor_dry[m³]'] = dfxls['vol_teor_cum_dry[m³]'].diff(periods=1).fillna(0).map(lambda x: 0 if x<0 else x).copy()

#### Calcolo di volumi dal Slurry-Feed * dt           ['dVol_bal_feed_slurry[m³]']

# dfxls['feed_adj[m³/h]'] = dfxls[col_list[pos_feed]]-dfxls[col_list[pos_pos02]]-dfxls[col_list[pos_pos03]]  # incasina non togliere i sensori 2 e 3
# dfxls['bal_feed_slurry[m³/h]']=dfxls[col_list[pos_slurry]]-dfxls['feed_adj[m³/h]']

dfxls['bal_feed_slurry[m³/h]']=dfxls[col_list[pos_slurry]]-dfxls[col_list[pos_feed]]

dfxls['dvol_bal_feed_slurry[m³]']=dfxls['bal_feed_slurry[m³/h]']*dfxls['datestamp_diff[h]']
# dfxls['dvol_bal_feed_slurry_cum[m³]']=dfxls['dvol_bal_feed_slurry[m³]'].cumsum().shift(-1).fillna(0)



#### Calcolo di volumi dal Slurry-Feed * dt   METHOD E

dfxls['rhosoil'] = rhosoil_
dfxls['rho_insitu'] = (dfxls.rhosoil *(1-dfxls.nw))+(dfxls.nw*rhow)

dfxls['dvol_calc_MthE[m³]']=(
                            (dfxls[col_list[pos_slurry]]*
                            (dfxls[col_list[pos_dens_slurry]]-1)/(dfxls.rhosoil-1))-
                            (dfxls[col_list[pos_feed]]*
                            (dfxls[col_list[pos_dens_feed]]-1)/(dfxls.rhosoil-1))
                            )*dfxls['datestamp_diff[h]']



# dfxls['dvol_calc_MthE_cum[m³]']=dfxls['dvol_calc_MthE[m³]'].cumsum().shift(-1).fillna(0)


#### Calcolo volumi dal fill level con formula polinomiale     ['dvol_work_chamb[m³]']

def calcvolume(x):
    vol=-0.113*x**5+0.1236*x**4+0.5462*x**3-0.1059*x**2+5.6112*x+11.794
    if x > 0 : 
        return vol
    elif x < 0 : 
        return vol
    else :
        return 0
    
dfxls['instant_volume[m³]'] = dfxls[col_list[pos_filllev]].map(lambda x: calcvolume(x))
dfxls['dvol_work_chamb[m³]'] = dfxls['instant_volume[m³]'].diff(periods=1).fillna(0).copy()
# dfxls['dvol_work_chamb_cum[m³]'] = dfxls['dvol_work_chamb[m³]'].cumsum().shift(-1).fillna(0)

#### Calcolo i balance istantanei e totali

dfxls['dvol_calc[m³]'] = dfxls['dvol_work_chamb[m³]']+dfxls['dvol_bal_feed_slurry[m³]']

dfxls['dvol_teor-calc[m³]'] = dfxls['dvol_teor[m³]']-dfxls['dvol_calc[m³]']





#### Creo la colonna STATE (fulloper,stall_rotating,stall_stand)

dfxls['state']=0
f_min_torq25 = dfxls.iloc[:,pos_torque].describe()[4]
f_min_adv25 = dfxls.iloc[:,pos_advspeed].describe()[4]

dfxls.loc[(dfxls[col_list[pos_torque]]>0) & (dfxls[col_list[pos_advspeed]]>0),'state'] = 'fulloper'
dfxls.loc[(dfxls[col_list[pos_torque]]!=0) & (dfxls[col_list[pos_advspeed]]==0),'state'] = 'stall_rotating'
dfxls.loc[(dfxls[col_list[pos_torque]]==0) & (dfxls[col_list[pos_advspeed]]==0),'state'] = 'stall_stand'

### CREATE col_list for fast individuation of columns

col_list=dfxls.columns.tolist()


### creo pos_dict per le colonne nuove

ls0 = []
ls1 = []

for i in col_list[111:]:
    
    ls0.append('pos_{}'.format(i.replace(" ",'').split("[")[0]))
    ls1.append(dfxls.columns.get_loc(i))
    
pos_dict = dict(zip(ls0,ls1))   

### Modifiche aggiuntive
- Correggo cambio concio colonna 'num_corr'
- Creo colonna **'change'** False-True quando passo da:
    - fermo( **'dstroke[mm]'** <=0 , **'state2'** =0)
    - in movimento( **'dstroke[mm]'** >0 , **'state2'** =1)
- Creo colonna **change** ovvero quando la macchina si ferma fz state2
- Crea colonna **change2** ovvero quando la macchina riparte fz state2
- Creo colonna **state3** basata su inizio fine processo
- Creo colonna **num_corr** che è il numero di concio corretto
- Creo colonna **'dstroke_pos[mm]'** e **'dstroke_pos_cum_seg[mm]'**
  per tenere conto del solo stroke di avanzamento per concio
- Calcolo i volumi cumulati:
    - **'dvol_teor_cum_seg[m³]'**
    - **'dvol_bal_feed_slurry_cum_seg[m³]'**
    - **'dvol_work_chamb_cum_seg[m³]'**
    - **'dvol_calc_cum_seg[m³]'**
    - **'dvol_teor-calc_cum_seg[m³]'**
    - **'dvol_calc_MthE_cum_seg[m³]'**
    - **theormass1** **theormass2**

In [ ]:
%%time
# creo colonna 'num_corr' 

dfxls.reset_index(drop=True)
for index, row in dfxls.iterrows():
    if index==0: numconcio = dfxls.iloc[0,pos_num]
    dfxls.loc[index,'num_corr'] = numconcio
    if index > 1 and row['dstroke[mm]']==0 and rowp['dstroke[mm]']>0 and row[col_list[pos_stroke]]>1500:
        numconcio += 1
        dfxls.loc[index,'num_corr'] = numconcio
    rowp = row


# creo colonna change in base a 'state2'

dfxls['change'] = dfxls['state2'].gt(dfxls['state2'].shift())
dfxls['change2'] = dfxls['state2'].shift().gt(dfxls['state2'])

dic_start = dfxls[dfxls['change']==True].groupby(col_list[pos_num]).agg({col_list[pos_timestamp]:min}).to_dict()
dic_start = dic_start[list(dic_start.keys())[0]]

dic_end = dfxls[dfxls['change2']==True].groupby(col_list[pos_num]).agg({col_list[pos_timestamp]:max}).to_dict()
dic_end = dic_end[list(dic_end.keys())[0]]


In [ ]:
%%time

# creo colonna change in base a 'state3'   Inizio fine processo di avanzamento senza tener conto di stop intermedi

def ffstate3(row):
    try:
        if row[col_list[pos_timestamp]]<dic_start[row['num_corr']]:
            return 0
    except:
        return 0
    try:
        if row[col_list[pos_timestamp]]>=dic_start[row['num_corr']]:
            return 1
    except:
        return 0
    try:
        if row[col_list[pos_timestamp]]>dic_end[row['num_corr']]:
            return 0
    except:
        return 0

dfxls['state3'] = 0
dfxls['state3'] = dfxls.apply(ffstate3, axis=1)

In [ ]:
%%time
######     RAGGRUPPO     PER  SEGMENT                                   ******* GROUP ********


    
gr = dfxls[dfxls.state3==1].groupby('num_corr')
# gr = dfxls.groupby('num_corr')

In [ ]:
%%time

dfxls['dstroke_pos_cum_seg[mm]'] = gr['dstroke_pos[mm]'].cumsum().fillna(0)
dfxls['dstroke_cum_seg[mm]'] = gr['dstroke[mm]'].cumsum().fillna(0)               # <---- è quello corretto perchè tiene conto di un possibile x0!=0
dfxls['dstroke_pos_cum_seg[mm]'].fillna(0)
dfxls['dstroke_cum_seg[mm]'].fillna(0, inplace=True)

## CALCOLO I VOLUMI CUMULATI    

dfxls['dvol_teor_cum_seg[m³]'] = gr['dvol_teor[m³]'].cumsum().fillna(0)                            # non fare shift
dfxls['dvol_teor_dry_cum_seg[m³]'] = gr['dvol_teor_dry[m³]'].cumsum().fillna(0)

dfxls['dvol_bal_feed_slurry_cum_seg[m³]'] = gr['dvol_bal_feed_slurry[m³]'].cumsum().fillna(0)     # non fare .shift(-1).fillna(0)
dfxls['dvol_work_chamb_cum_seg[m³]'] = gr['dvol_work_chamb[m³]'].cumsum().fillna(0)
dfxls['dvol_calc_cum_seg[m³]'] = gr['dvol_calc[m³]'].cumsum().fillna(0)
dfxls['dvol_teor-calc_cum_seg[m³]'] = gr['dvol_teor-calc[m³]'].cumsum().fillna(0)
dfxls['dvol_calc_MthE_cum_seg[m³]'] = gr['dvol_calc_MthE[m³]'].cumsum().fillna(0)


### DRY MASS CONTROL PARAMETERS

 1. Teoretical Mass (Stroke*Area*Density insitu)  **'theormass1'**
 2. Calc dry mass A(particle+bentonite)           **'metha'**
 3. Calc dry mass B(gener solid + fixed water)    **'methb'**
 4. Calc dry mass C(insitu density)               **'methc'**
 5. Calc dry mass D(particle + insitu density)    **'methd'**


In [ ]:
##### Theoretical mass


dfxls['dtheormass2'] = Area_Cutter * dfxls.rho_insitu *(dfxls[cl[pos_advspeed]]*60/1000*dfxls['datestamp_diff[h]'])[dfxls.state3==1] 
dfxls['dtheormass2'].fillna(0)

dfxls['metha1'] = dfxls[cl[pos_feed]]*rhobent*(dfxls[cl[pos_dens_feed]]-rhow)/(rhobent-rhow)
dfxls['metha2'] = dfxls[cl[pos_slurry]]*dfxls.rhosoil*(dfxls[cl[pos_dens_slurry]]-rhow)/(dfxls.rhosoil-rhow)
dfxls['metha3'] = (dfxls.metha2 - dfxls.metha1)*dfxls.dt_h[dfxls.state3==1] 


dfxls['methb1'] = ((dfxls[cl[pos_slurry]]*(dfxls[cl[pos_dens_slurry]]-rhow))-(dfxls[cl[pos_feed]]*(dfxls[cl[pos_dens_feed]]-rhow)))*dfxls.rhosoil / (dfxls.rhosoil-rhow)*dfxls.dt_h


dfxls['methc1'] = ((dfxls[cl[pos_slurry]]*dfxls[cl[pos_dens_slurry]])-(dfxls[cl[pos_feed]]*dfxls[cl[pos_dens_feed]]))/(dfxls.rho_insitu - dfxls[cl[pos_dens_feed]])*dfxls.dt_h


dfxls['methd1'] = ((dfxls[cl[pos_slurry]]*(dfxls[cl[pos_dens_slurry]]-rhow))-(dfxls[cl[pos_feed]]*(dfxls[cl[pos_dens_feed]]-rhow)))/(dfxls.rho_insitu - dfxls[cl[pos_dens_feed]])*dfxls.dt_h
dfxls['methd2'] = (dfxls.methd1 * dfxls.rhosoil/(dfxls.rhosoil-rhow))+dfxls['dvol_work_chamb[m³]']*dfxls[cl[pos_dens_slurry]]



######     RAGGRUPPO     PER  SEGMENT                                   ******* GROUP ********

    
gr = dfxls[dfxls.state3==1].groupby('num_corr')
# gr = dfxls.groupby('num_corr')


dfxls['theormass1'] = Area_Cutter * dfxls.rho_insitu * dfxls['dstroke_pos_cum_seg[mm]']/1000
dfxls['theormass2'] = gr['dtheormass2'].cumsum().fillna(0)
dfxls['metha'] = gr.metha3.cumsum().fillna(0)
dfxls['methb'] = gr.methb1.cumsum().fillna(0)
dfxls['methc'] = gr.methc1.cumsum().fillna(0)
dfxls['methd'] = gr.methd2.cumsum().fillna(0)

### FACE PRESSURE CONTROL PARAMETERS

In [ ]:
## Posizione dei sensori

z_s8 = 2.9
z_s17 = 1.8
z_s2 = 0.3
z_s36 = -0.6
z_s45 = -1.9

z_s1 = z_s17
z_s7 = z_s17
z_s3 = z_s36
z_s6 = z_s36
z_s4 = z_s45
z_s5 = z_s45

x_s1 = 1.664
x_s2 = 2.031
x_s3 = 3.033
x_s4 = 1.628
x_s5 = -1.608
x_s6 = -2.95
x_s7 = -1.737
x_s8 = -0.867


dfxls['rhopat'] = 1.25


dfxls['paau_s8'] = 2.36
dfxls['pau_s8'] = 1.98
dfxls['pn_s8']   = 1.8
dfxls['pal_s8'] = 1.71 
dfxls['paal_s8'] = 1.62

lista_sens = ['s17','s2','s36','s45']

for i in lista_sens:
       
    dfxls[('paau_{}').format(i)] = dfxls['paau_s8'] + dfxls[cl[pos_dens_p04]] * (z_s8-vars()[('z_{}').format(i)])/10
    dfxls[('pau_{}').format(i)] = dfxls['pau_s8'] + dfxls[cl[pos_dens_p04]] * (z_s8-vars()[('z_{}').format(i)])/10
    dfxls[('pn_{}').format(i)] = dfxls['pn_s8'] + dfxls[cl[pos_dens_p04]] * (z_s8-vars()[('z_{}').format(i)])/10
    dfxls[('pal_{}').format(i)] = dfxls['pal_s8'] + dfxls[cl[pos_dens_p04]] * (z_s8-vars()[('z_{}').format(i)])/10
    dfxls[('paal_{}').format(i)] = dfxls['paal_s8'] + dfxls[cl[pos_dens_p04]] * (z_s8-vars()[('z_{}').format(i)])/10
    


    
    
    
    
dfxls[['paal_s36','paau_s36','pal_s36','pau_s36','pn_s36','pn_s8']]







### RISK LEVEL PER SENSOR CHAMBER

In [ ]:
lista_sens = ['s1','s2','s3','s4','s5','s6','s7','s8']




### risk 
# 4: upper alarm level passed
# 3: upper alert level passed
# 2: normal
# 1: lower alert level passed
# 0: lower alarm level passed
        
    
    
dfxls['risk_s1'] = dfxls.apply(lambda x: 4 if x[cl[pos_prch01]]>x['paau_s17'] else (
                               3 if x[cl[pos_prch01]]>x['pau_s17'] else (
                               0 if x[cl[pos_prch01]]<x['paal_s17'] else (
                               1 if x[cl[pos_prch01]]<x['pal_s17'] else 2))),
                               axis=1)  

dfxls['risk_s2'] = dfxls.apply(lambda x: 4 if x[cl[pos_prch02]]>x['paau_s2'] else (
                               3 if x[cl[pos_prch02]]>x['pau_s2'] else (
                               0 if x[cl[pos_prch02]]<x['paal_s2'] else (
                               1 if x[cl[pos_prch02]]<x['pal_s2'] else 2))),
                               axis=1) 

dfxls['risk_s3'] = dfxls.apply(lambda x: 4 if x[cl[pos_prch03]]>x['paau_s36'] else (
                               3 if x[cl[pos_prch03]]>x['pau_s36'] else (
                               0 if x[cl[pos_prch03]]<x['paal_s36'] else (
                               1 if x[cl[pos_prch03]]<x['pal_s36'] else 2))),
                               axis=1) 

dfxls['risk_s4'] = dfxls.apply(lambda x: 4 if x[cl[pos_prch04]]>x['paau_s45'] else (
                               3 if x[cl[pos_prch04]]>x['pau_s45'] else (
                               0 if x[cl[pos_prch04]]<x['paal_s45'] else (
                               1 if x[cl[pos_prch04]]<x['pal_s45'] else 2))),
                               axis=1) 

dfxls['risk_s5'] = dfxls.apply(lambda x: 4 if x[cl[pos_prch05]]>x['paau_s45'] else (
                               3 if x[cl[pos_prch05]]>x['pau_s45'] else (
                               0 if x[cl[pos_prch05]]<x['paal_s45'] else (
                               1 if x[cl[pos_prch05]]<x['pal_s45'] else 2))),
                               axis=1) 

dfxls['risk_s6'] = dfxls.apply(lambda x: 4 if x[cl[pos_prch06]]>x['paau_s36'] else (
                               3 if x[cl[pos_prch06]]>x['pau_s36'] else (
                               0 if x[cl[pos_prch06]]<x['paal_s36'] else (
                               1 if x[cl[pos_prch06]]<x['pal_s36'] else 2))),
                               axis=1) 

dfxls['risk_s7'] = dfxls.apply(lambda x: 4 if x[cl[pos_prch07]]>x['paau_s17'] else (
                               3 if x[cl[pos_prch07]]>x['pau_s17'] else (
                               0 if x[cl[pos_prch07]]<x['paal_s17'] else (
                               1 if x[cl[pos_prch07]]<x['pal_s17'] else 2))),
                               axis=1) 

dfxls['risk_s8'] = dfxls.apply(lambda x: 4 if x[cl[pos_prch08]]>x['paau_s8'] else (
                               3 if x[cl[pos_prch08]]>x['pau_s8'] else (
                               0 if x[cl[pos_prch08]]<x['paal_s8'] else (
                               1 if x[cl[pos_prch08]]<x['pal_s8'] else 2))),
                               axis=1) 

### DENS CALC

In [ ]:
dfxls.d87 = (dfxls[cl[pos_prch07]]-dfxls[cl[pos_prch08]])/abs(z_s7-z_s8)*10
dfxls.d81 = (dfxls[cl[pos_prch01]]-dfxls[cl[pos_prch08]])/abs(z_s1-z_s8)*10

dfxls.d82 = (dfxls[cl[pos_prch02]]-dfxls[cl[pos_prch08]])/abs(z_s2-z_s8)*10

dfxls.d76 = (dfxls[cl[pos_prch06]]-dfxls[cl[pos_prch07]])/abs(z_s6-z_s7)*10
dfxls.d13 = (dfxls[cl[pos_prch03]]-dfxls[cl[pos_prch01]])/abs(z_s3-z_s1)*10

dfxls.d76 = (dfxls[cl[pos_prch05]]-dfxls[cl[pos_prch06]])/abs(z_s5-z_s6)*10
dfxls.d73 = (dfxls[cl[pos_prch04]]-dfxls[cl[pos_prch03]])/abs(z_s4-z_s3)*10

### Lista delle variabili create

In [ ]:
### Lista delle variabili create

# 'vol_teor_cum[m³]'         # usare 'dvol_teor_cum_seg[m³]'
# 'dvol_teor[m³]'
# 'dvol_bal_feed_slurry[m³]'    
# 'dvol_work_chamb[m³]'
# 'dvol_calc[m³]'
# 'dvol_calc_MthE[m³]'
# 'dvol_teor-calc[m³]'
# 'dvol_teor_cum_seg[m³]'
# 'dvol_bal_feed_slurry_cum_seg[m³]'
# 'dvol_work_chamb_cum_seg[m³]'
# 'dvol_calc_cum_seg[m³]'
# 'dvol_teor-calc_cum_seg[m³]'
# 'dvol_calc_MthE_cum_seg[m³]'

# 'theormass'
# 'metha'
# 'methb'
# 'methc'
# 'methd'

# paal_s36
# paau_s36
# pal_s36
# pau_s36
# pn_s36

# z_s1
# x_s1




# dfxls.d87
# dfxls.d81
# dfxls.d82
# dfxls.d76
# dfxls.d13
# dfxls.d76
# dfxls.d73

## PLOT Get Ready to run

In [ ]:
dfxlscopy = dfxls.copy()     

In [ ]:
# segment 1859-1931

rigainizio = 0  # 0    per partire dalla prima  riga inziale
rigafine = -1     #-1   riga finale 
dstep    = 1     #  1   detaspet uguale al db 30sec o 10sec

df1 = dfxlscopy.iloc[rigainizio:rigafine,:].reset_index(drop=True)

n_row = df1.shape[0]
n_row

In [ ]:
# creo una lista con data, sensori delle camere, fill, 
list_chamb = []
for i in dfxls.filter(regex='Pressure excavation chamber'):
    list_chamb.append(dfxls.columns.get_loc(i))
print('List of chamber columns is:',list_chamb)

list_col = [0]
list_col.extend(list_chamb)
list_col.append(pos_filllev)
list_col.extend([dfxls.columns.get_loc('risk_s1'),dfxls.columns.get_loc('risk_s2'),dfxls.columns.get_loc('risk_s3'),dfxls.columns.get_loc('risk_s4'),
               dfxls.columns.get_loc('risk_s5'),dfxls.columns.get_loc('risk_s6'),dfxls.columns.get_loc('risk_s7'),dfxls.columns.get_loc('risk_s8')])
print('List of interesting columns',list_col)

# *************************************************

n_sensori = 8

list_x = [x_s1,x_s2,x_s3,x_s4,x_s5,x_s6,x_s7,x_s8]
list_z = [z_s1,z_s2,z_s3,z_s4,z_s5,z_s6,z_s7,z_s8]

# zeta sfalsate per plot 
list_z1 = [z_s1-0.5,z_s2,z_s3-0.5,z_s4,z_s5-0.5,z_s6,z_s7,z_s8]

print(list_x)
print(list_z)

# list_x = []
# list_z = []
# for i in np.arange(0,n_sensori,1):
#     x = 1 * np.sin(2*np.pi/8*i)
#     list_x.append(x)
#     z = 1 * np.cos(2*np.pi/8*i)
#     list_z.append(z)


    
df2 = dfxlscopy.iloc[rigainizio:rigafine,list_col].copy().reset_index(drop=True)

In [ ]:
#  ************************************************

plt.style.use('dark_background')

save_folder = 'images/gif'
gif_filename = '01'

shutil.rmtree(save_folder, ignore_errors=True)

working_folder = '{}/{}'.format(save_folder, gif_filename)

if not os.path.exists(working_folder):
    os.makedirs(working_folder)
    
    
fontdicttitlebig={'fontsize': 20,
          'fontweight':0.1,
          'horizontalalignment': 'center'}    

fontdict1={'fontsize': 18,
          'fontweight':0.1,
          'horizontalalignment': 'center'}

fontdict2 = {'family': 'arial',
        'color':  'white',
        'weight': 'normal',
        'size': 16,
        }

font_tx_graph = {'family': 'arial',
        'color':  'white',
        'weight': 'normal',
        'size': 14,
        }

colorstate = {'fulloper':'limegreen','stall_rotating':'orange','stall_stand':'red'}
textstate = {'fulloper':'OPERATION','stall_rotating':'ONLY ROTATING','stall_stand':'STOP'}
colorfilllev = {True:'r',False:'b'}
colorbalfs = {True:'r',False:'b'}

colorrisk = {0:'tomato',1:'yellow',2:'springgreen',3:'yellow',4:'tomato'}

cmap_segment = cm.get_cmap('jet')




# Inizializzo le variabili indicatore


n_sensori = 8



changepoint = 0

nseg = 0
nseg_p = 0


p_ch8_max = 0
p_ch7_max = 0
p_ch6_max = 0
p_ch5_max = 0
p_ch4_max = 0
p_ch3_max = 0
p_ch2_max = 0
p_ch1_max = 0

stroke = 0

vol_teor_cum = 0
vol_teor_cum_p = 0

stroke_cum = 0
stroke_cum_p = 0

bal_fs_sum = 0
bal_fs_sum_p = 0
bal_fs_q = 0

vol_wc_cum = 0
vol_wc_cum_p = 0

advdsp = []
date = []

nframe = 0

balancegrout = 0
balancetot = 0


###############################################################################################################################


for e in np.arange(0,n_row,1):
    
    if e%dstep==0:    # creo immagine ogni dstep
        
        try:
            ax.clear()
            ax0.clear()
            ax1.clear()
            ax2.clear()
            ax3.clear()
            ax4.clear()
            ax5.clear()
            ax6.clear()
            
            
        except:
            pass
        
        
        fig = plt.figure(figsize=(27,23))
        
        
        gs = fig.add_gridspec(13, 3)
        
        
        ##################################  PRIMO GRAFICO


        ax0 = fig.add_subplot(gs[0:2, :])
               

        ## 1 rett
        
        if df1['num_corr'][e]!= nseg: 
            nseg_p = nseg
            changepoint = 1
            
        if df1['num_corr'][e] == nseg: changepoint = 0
               
        nseg = df1['num_corr'][e]
        
        ## 2 rett

        p_ch8_act = df1.iloc[e,pos_prch08]
        p_ch8_max = max(p_ch8_max,df1.iloc[e,pos_prch08])
        
        p_ch7_act = df1.iloc[e,pos_prch07]
        p_ch7_max = max(p_ch7_max,df1.iloc[e,pos_prch07]) 
        
        p_ch6_act = df1.iloc[e,pos_prch06]
        p_ch6_max = max(p_ch6_max,df1.iloc[e,pos_prch06])
        
        p_ch5_act = df1.iloc[e,pos_prch05]
        p_ch5_max = max(p_ch5_max,df1.iloc[e,pos_prch05])
        
        p_ch4_act = df1.iloc[e,pos_prch04]
        p_ch4_max = max(p_ch4_max,df1.iloc[e,pos_prch04])
        
        p_ch3_act = df1.iloc[e,pos_prch03]
        p_ch3_max = max(p_ch3_max,df1.iloc[e,pos_prch03])
        
        p_ch2_act = df1.iloc[e,pos_prch02]
        p_ch2_max = max(p_ch2_max,df1.iloc[e,pos_prch02])
        
        p_ch1_act = df1.iloc[e,pos_prch01]
        p_ch1_max = max(p_ch1_max,df1.iloc[e,pos_prch01])
        
        
        ### 3 rett
        
        feedl = df1.iloc[e,pos_feed]
        slurry = df1.iloc[e,pos_slurry]
        
        bal_fs_q = slurry - feedl
        
        bal_fs = df1['dvol_bal_feed_slurry[m³]'][e]
         
        bal_fs_sum = df1['dvol_bal_feed_slurry_cum_seg[m³]'][e]
        
        
        ### 4 rect
        
        fillev = df1.iloc[e,pos_filllev]
        
        if e==0: fillev_p = 0
        if e!=0: fillev_p = df1.iloc[e-1,pos_filllev]
        
        
        dfillev = fillev-fillev_p
        
        dvolwch = df1['dvol_work_chamb[m³]'][e]
        vol_wc_cum = df1['dvol_work_chamb_cum_seg[m³]'][e]
        
        ### 5 rect
        
        stroke = df1['dstroke_pos[mm]'][e]       
        stroke_cum = df1['dstroke_cum_seg[mm]'][e]
        
        dvolteor = df1['dvol_teor[m³]'][e]
        vol_teor_cum = df1['dvol_teor_cum_seg[m³]'][e]
        
        
        ###  6 rett 
        
        
        if changepoint == 1 : 
            balancegrout_p = balancegrout
            balancetot_p = balancetot
        
        
        balancegrout = df1['dvol_calc_cum_seg[m³]'][e]
        balancetot = df1['dvol_teor-calc_cum_seg[m³]'][e]
        
            
        rectangles = {'Info1' : mpl.patches.Rectangle((0,1), 4, 2, color=cmap_segment(nseg%10)),
              'Info2'  : mpl.patches.Rectangle((0,4), 4, 6, color=colorstate[df1.state[e]]),
                      
#             'Info3'   : mpl.patches.Rectangle((5,1), 4, 2, color='grey'),
#             'Info4'  : mpl.patches.Rectangle((5,4), 4, 6, color='grey'),
                      
              'Info3' : mpl.patches.Rectangle((5,1), 4, 2, color=colorbalfs[bal_fs<0]),
              'Info4'  : mpl.patches.Rectangle((5,4), 4, 6, color=colorbalfs[bal_fs_q<0]),
                      
              'Info5'   : mpl.patches.Rectangle((10,1), 4, 2, color=colorbalfs[vol_wc_cum<0]),
              'Info6'  : mpl.patches.Rectangle((10,4), 4, 6, color=colorbalfs[bal_fs<0]),
                      
              'Info7'   : mpl.patches.Rectangle((15,1), 4, 4, color=colorstate[df1.state[e]]),
              'Info8'  : mpl.patches.Rectangle((15,6), 4, 4, color=colorstate[df1.state[e]]),
              
              'Info9'   : mpl.patches.Rectangle((20,1), 4, 4, color=colorbalfs[balancegrout<0]),
              'Info10'  : mpl.patches.Rectangle((20,6), 4, 4, color=colorbalfs[balancegrout<0]),              
                      
              'Info11'   : mpl.patches.Rectangle((25,1), 4, 4, color=colorbalfs[balancetot<0]),
              'Info12'  : mpl.patches.Rectangle((25,6), 4, 4, color=colorbalfs[balancetot_p<0])}
        
        
        dict1 = {'Info1': 'Segment {}'.format(int(nseg)),
                 'Info2': '{}'.format(df1.state[e]),
                 
#                'Info3': 'P ch8:{} bar\nP ch5:{} bar'.format(p_ch8_act,p_ch5_act),
#                'Info4': 'Max P ch8:\n{} bar\nMax P ch5:\n{} bar'.format(p_ch8_max,p_ch5_max),
                 
                 'Info3': 'Volcum:{:.2f}[m³]'.format(bal_fs_sum),
                 'Info4': 'Feed:\n{:.2f}[m³/h]\nSlurry\n{:.2f}[m³/h]\nDiff:{:.1f}[m³/h]\nDiff:{:.1f}[m³]'.format(feedl,slurry,bal_fs_q,bal_fs),
                 
                 'Info5': 'Volcum:{:.2f}[m³]'.format(vol_wc_cum),
                 'Info6': 'Fill level:\n{:.2f}[m]\nFill lev pre\n{:.2f}[m]\nDiff:{:.1f}[m]\nDiff:{:.1f}[m³]'.format(fillev,fillev_p,dfillev,dvolwch),
                 
                 'Info7': 'Stroke:{:.2f}[mm]\n Volcum:{:.2f}[m³]'.format(stroke_cum, vol_teor_cum),
                 'Info8': 'dStroke:{:.2f}[mm]\n dVol:{:.2f}[m³]'.format(stroke , dvolteor), 
                 
                 'Info9': 'Balance \n grout',
                 'Info10': 'Bal grout\n:{:.2f}[m³]'.format(balancegrout),                     
                 
                 'Info11': 'BAL TOT\n:{:.2f}[m³]'.format(balancetot),
                 'Info12': 'Prev segm:{}[-]\n BAL TOT PREV:\n{:.2f}[m³]'.format(nseg_p, balancetot_p),
                }


        ##########################################################################################
        ##################################  RECTANGLES INFO ****************************************
        ##########################################################################################
        


        for r in rectangles:
            ax0.add_artist(rectangles[r])
            rx, ry = rectangles[r].get_xy()
            cx = rx + rectangles[r].get_width()/2.0
            cy = ry + rectangles[r].get_height()/2.0

            ax0.annotate(dict1[r], (cx, cy), color='w', weight='bold', 
                        fontsize=15, ha='center', va='center')
            
    
        

        ax0.set_xlim((0, 30))
        ax0.set_ylim((0, 11))
        # ax0.set_aspect('equal')

        ax0.axis('off')
        
        ax0.set_title(str(df1.iloc[e,pos_date]), fontsize = 20.0, pad=0)
        
        
        ##########################################################################################
        ##################################  PRIMO GRAFICO ****************************************
        ##########################################################################################
        
        
        
        ax = fig.add_subplot(gs[3:6, 1], projection='3d')
        ax1 = fig.add_subplot(gs[3:6, 2], projection='3d')
        ax4 = fig.add_subplot(gs[3:6, 0])


        ax.set_xlabel("x")
        ax.set_ylabel("y") 
        ax.set_zlabel("z")

        ax.view_init(45, -45) # rotazione della vista 3d


        
        # Strech the graph
        ax.get_proj = lambda: np.dot(Axes3D.get_proj(ax), np.diag([1.0, 1.1, 1.0, 1]))

        
        
        #######   GRAFICO A SINISTRA e CENTRALE PRESSURE CHAMBERS  *********


        ax.set_xlabel('x'),ax.set_ylabel('Pressure [bar]'),ax.set_zlabel('y')

        n_row = df2.shape[0]

        for i in np.arange(0,n_sensori,1):

            x_ = list_x[i]
            z_ = list_z1[i]
            y_ = df2.iloc[e,i+1]
            c_ = df2.iloc[e,i+10] # fare riga colore per il grafico a barre
             

            # Plotto una barra per volta
            ax.bar3d(x_, y_, z_, dx=0.8, dz=0.8, dy=(0-y_),
                    color='deepskyblue')
            
            ax4.barh(z_, y_, align='center', height=0.4, color=colorrisk[c_])
            
            ax4.set_xlabel('Pressure [bar]'),ax4.set_ylabel('Distance from center[m]')
            ax4.set_title('Pressure [bar]')
            ax4.set_xticks
            
#             ax4.set_yticks
#             ax4.set_yticklabels(people)
#             
#             ax4.set_xlabel('Performance')
#             
            
        ax4.plot((df1.paau_s8[e],df1.paau_s45[e]),(z_s8,z_s4), c='r')
        ax4.plot((df1.pau_s8[e],df1.pau_s45[e]),(z_s8,z_s4), c='orange')
        ax4.plot((df1.pn_s8[e],df1.pn_s45[e]),(z_s8,z_s4), c='limegreen')
        ax4.plot((df1.pal_s8[e],df1.pal_s45[e]),(z_s8,z_s4), c= 'orange')
        ax4.plot((df1.paal_s8[e],df1.paal_s45[e]),(z_s8,z_s4), c='r')
        
        ax4.invert_xaxis()  # labels read top-to-bottom            
            
        x1_ = 0
        z1_ = 0
        y1_ = df1.iloc[e, pos_p02]/100
            
        ax.bar3d(x1_, y1_, z1_, dx=0.9, dz=0.9, dy=(0-y1_), color='red')    
        
        ax.set_xlim3d([-3.6,3.6])
        ax.set_ylim3d([3,0]) # same as saying to invert it 
        ax.set_zlim3d([-3.6,3.6]) 
        
        # livelli di pressione massimi barre magenta
        
        ax4.plot([p_ch8_max,p_ch8_max], [2.75,3.05], lw=4, c='m')
        ax4.plot([p_ch7_max,p_ch7_max], [1.65,1.95], lw=4, c='m')
        ax4.plot([p_ch1_max,p_ch1_max], [1.15,1.45], lw=4, c='m')
        ax4.plot([p_ch2_max,p_ch2_max], [0.15,0.45], lw=4, c='m')
        ax4.plot([p_ch6_max,p_ch6_max], [-0.75,-0.45], lw=4, c='m')
        ax4.plot([p_ch3_max,p_ch6_max], [-1,-1.3], lw=4, c='m')
        ax4.plot([p_ch4_max,p_ch4_max], [-2.05,-1.75], lw=4, c='m')
        ax4.plot([p_ch5_max,p_ch5_max], [-2.55,-2.25], lw=4, c='m')
        
        ax4.text(p_ch8_max, 2.8, '{:.2f} '.format(p_ch8_max),horizontalalignment='right', fontdict = font_tx_graph)  
        ax4.text(p_ch7_max, 1.8, '{:.2f} '.format(p_ch7_max),horizontalalignment='right', fontdict = font_tx_graph)
        ax4.text(p_ch1_max, 1.3, '{:.2f} '.format(p_ch1_max),horizontalalignment='right', fontdict = font_tx_graph)
        ax4.text(p_ch2_max, 0.2, '{:.2f} '.format(p_ch2_max),horizontalalignment='right', fontdict = font_tx_graph)
        ax4.text(p_ch6_max, -0.7, '{:.2f} '.format(p_ch6_max),horizontalalignment='right', fontdict = font_tx_graph)
        ax4.text(p_ch3_max, -1.2, '{:.2f} '.format(p_ch3_max),horizontalalignment='right', fontdict = font_tx_graph)
        ax4.text(p_ch4_max, -2.1, '{:.2f} '.format(p_ch4_max),horizontalalignment='right', fontdict = font_tx_graph)    
        ax4.text(p_ch5_max, -2.6, '{:.2f} '.format(p_ch5_max),horizontalalignment='right', fontdict = font_tx_graph) 
        
        ax4.text(-0, 2.8, ' C8 {:.2f} bar'.format(df1[cl[pos_prch08]][e]), fontdict = font_tx_graph)  
        ax4.text(-0, 1.8, ' C7 {:.2f} bar'.format(df1[cl[pos_prch07]][e]), fontdict = font_tx_graph)
        ax4.text(-0, 1.3, ' C1 {:.2f} bar'.format(df1[cl[pos_prch01]][e]), fontdict = font_tx_graph)
        ax4.text(-0, 0.2, ' C2 {:.2f} bar'.format(df1[cl[pos_prch02]][e]), fontdict = font_tx_graph)
        ax4.text(-0, -0.7, ' C6 {:.2f} bar'.format(df1[cl[pos_prch06]][e]), fontdict = font_tx_graph)
        ax4.text(-0, -1.2, ' C3 {:.2f} bar'.format(df1[cl[pos_prch03]][e]), fontdict = font_tx_graph)
        ax4.text(-0, -2.1, ' C4 {:.2f} bar'.format(df1[cl[pos_prch04]][e]), fontdict = font_tx_graph)
        ax4.text(-0, -2.6, ' C5 {:.2f} bar'.format(df1[cl[pos_prch05]][e]), fontdict = font_tx_graph)
            
        ############################################
        # Plotto la barra rossa FILL LEVEL    ********************************************************
        
        z1_ = df1.iloc[e,pos_filllev]
        
        ax1.bar3d(-1, 0, z1_, dx=2, dy=0.5, dz=(-2-z1_),
                    color = colorfilllev[df1.iloc[e,pos_filllev]<-1.8])  


        x1_ = 0.5
        z1_ = 3
        y1_ = df1.iloc[e, pos_p01]/100
        ax1.bar3d(x1_, y1_, z1_, dx=0.2, dz=0.2, dy=(0-y1_), color='red')       
        
        x1_ = -0.5
        z1_ = 3
        y1_ = df1.iloc[e, pos_p05]/100
        ax1.bar3d(x1_, y1_, z1_, dx=0.2, dz=0.2, dy=(0-y1_), color='red')  
        
        ax1.set_xlim3d(-1,1)
        ax1.set_ylim3d(2,0) # same as saying to invert it 
        ax1.set_zlim3d(-2,3) 
        
        
        
        # ax0.text(0.05, 0.9, str(df2.iloc[e,0]), transform=ax.transAxes, fontdict = fontdict2)  ## data ora
        ax1.text2D(0, -0.3, str('Pressure chamber[bar]\nP0.2[m³/h/100]'), transform=ax.transAxes, fontdict = fontdict2)    
        ax1.text2D(-1.2, -0.3, str('Pressure chamber[bar]\nLimit boundaries[bar]'), transform=ax.transAxes, fontdict = fontdict2)
        ax1.text2D(1.5, -0.3, str('Fill Level[m]\nP0.1,P0.5[m³/h/100]'), transform=ax.transAxes, fontdict = fontdict2)       

        
        
        ##################################  CONTOUR PLOT ##################################################
        
        ## creo data
        
                      ## data pressioni
        x = list_x
        y = list_z
        
        
        
        z = [df1[cl[pos_prch01]][e],
             df1[cl[pos_prch02]][e],
             df1[cl[pos_prch03]][e],
             df1[cl[pos_prch04]][e],
             df1[cl[pos_prch05]][e],
             df1[cl[pos_prch06]][e],
             df1[cl[pos_prch07]][e],
             df1[cl[pos_prch08]][e]]
        
                     ## data densità
        
        x1 = (x[0]+x[7])/2
        y1 = (y[0]+y[7])/2
        z1 = (z[0]-z[7])/(y[7]-y[0])*10

        x2 = (x[6]+x[7])/2
        y2 = (y[6]+y[7])/2
        z2 = (z[6]-z[7])/(y[7]-y[6])*10

        x3 = (x[5]+x[6])/2
        y3 = (y[5]+y[6])/2
        z3 = (z[5]-z[6])/(y[6]-y[5])*10

        x4 = (x[4]+x[5])/2
        y4 = (y[4]+y[5])/2
        z4 = (z[4]-z[5])/(y[5]-y[4])*10

        x5 = (x[3]+x[2])/2
        y5 = (y[3]+y[2])/2
        z5 = (z[3]-z[2])/(y[2]-y[3])*10

        x6 = (x[2]+x[1])/2
        y6 = (y[2]+y[1])/2
        z6 = (z[2]-z[1])/(y[1]-y[2])*10

        x7 = (x[1]+x[0])/2
        y7 = (y[1]+y[0])/2
        z7 = (z[1]-z[0])/(y[0]-y[1])*10


        xd = [x1,x2,x3,x4,x5,x6,x7]
        yd = [y1,y2,y3,y4,y5,y6,y7]
        zd = [z1,z2,z3,z4,z5,z6,z7]
      
        
        
        
        #######  1
        
        ax5 = fig.add_subplot(gs[7:10, 0])   

        grid_x, grid_y = np.mgrid[-3.1:3.1:200j, -3.1:3.1:200j]
        grid_z2 = griddata((x,y), z, (grid_x, grid_y), method='cubic')

        plt.imshow(grid_z2.T, extent=(-3.1,3.1,-3.1,3.1), cmap=plt.cm.rainbow, vmin=df1.paal_s45[e], vmax=df1.paau_s8[e],origin='lower')
        plt.colorbar()
        
        circle1 = plt.Circle((0, 0), 3.1, color=colorstate[df1.state[e]], lw = 5, fill=False)
        ax5.add_artist(circle1)
        
        plt.text(-1.2, -1.7, str('Contour plot pressure chamber[bar]'), transform=ax.transAxes, fontdict = fontdict2)
        

        #######  2
        
        ax6 = fig.add_subplot(gs[7:10, 1])
        
        grid_x, grid_y = np.mgrid[-3.1:3.1:200j, -3.1:3.1:200j]
        grid_z3 = griddata((xd,yd), zd, (grid_x, grid_y), method='cubic')

        plt.imshow(grid_z3.T, extent=(-3.1,3.1,-3.1,3.1), cmap=plt.cm.rainbow,vmin=1, vmax=1.4,origin='lower')
        plt.colorbar()

        circle1 = plt.Circle((0, 0), 3.1, color=colorstate[df1.state[e]], lw = 5, fill=False)
        ax6.add_artist(circle1)

        plt.text(0, -1.7, str('Contour plot density [t/m³]'), transform=ax.transAxes, fontdict = fontdict2)          
        
        
        ##################################  ULTIMO GRAFICO TIME DOMAIN ##################################################
        
        
        
        ax3 = fig.add_subplot(gs[11:13, :])
        
        
        advdsp.append(df1['dstroke_cum_seg[mm]'][e])
        date.append(df1['datestamp'][e])
        
        y = np.array(advdsp, dtype='int')
        x = np.array(date, dtype='datetime64')
        
        ax3.plot(x,y, label='stroke_cum_seg[mm]')
        
        # ax3.legend()
        
        plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%m-%d %H:%M'))
        
        # plt.gca().xaxis.set_major_locator(mdates.DayLocator())
        # ax3.xticks(rotation=70)
        # plt.gcf().autofmt_xdate()
        
        ax3.set_ylim(ymin = 0, ymax=1600)
        
        startdate = df1.datestamp[e] - timedelta(hours=1)
        enddate = df1.datestamp[e] + timedelta(hours=1)    
        
        ax3.set_xlim(xmin=startdate,xmax=enddate)
        
        
        
        plt.text(-1.2, -2.7, str('Stroke [mm] vs time'), transform=ax.transAxes, fontdict = fontdict2)


        ##################################  SALVA IMMAGINE ##################################################
        
        nframe +=1
        
        nomepng = '{}/{}/img_{:04d}.png'.format(save_folder, gif_filename, nframe)
        plt.savefig(nomepng) 
        
        # trick: i use a special name to get the file always updated otherwise the picture is not always the last created
        nomepng1 = '{}/{}/img_{:04d}.png?{}'.format(save_folder, gif_filename,nframe,randint(1, 100))   # ?1-100 random  serve per permettere l'update
        
    
        # don't display the static plot...
        plt.close()

      
IPdisplay.Image(url=nomepng1)    

In [ ]:
# load all the static images into a list then save as an animated gif

gif_filepath = '{}/{}.gif'.format(save_folder, gif_filename)


images = []
images = [Image.open(image) for image in glob.glob('{}/*.png'.format(working_folder))]
gif = images[0]
gif.info['duration'] = 400 #milliseconds per frame
gif.info['loop'] = 0 #how many times to loop (0=infinite)
append_images=[]
gif.save(fp=gif_filepath, format='gif', save_all=True, append_images=images[1:])


gif_filepath = '{}/{}.gif?{}'.format(save_folder, gif_filename,randint(1, 100))   # ?1  serve per permettere l'update
IPdisplay.Image(url=gif_filepath)

In [ ]:
col_list_print = [cl[pos_stroke],cl[pos_advspeed],'dstroke_pos[mm]','dstroke_pos_cum_seg[mm]','num_corr','change','change2','state','state1','state2','state3',cl[pos_filllev],cl[pos_feed],cl[pos_slurry],'vol_teor_cum[m³]','dvol_teor[m³]','dvol_teor_dry[m³]','bal_feed_slurry[m³/h]','dvol_bal_feed_slurry[m³]','dvol_work_chamb[m³]','dvol_calc[m³]','dvol_calc_cum_seg[m³]','dvol_calc_MthE[m³]','dvol_teor_cum_seg[m³]','dvol_calc_MthE_cum_seg[m³]','dvol_teor-calc[m³]']
# col_list_print =  [cl[pos_stroke], 'vol_teor_cum[m³]','dvol_teor[m³]']

# col_list_print = ['state3','num_corr','theormass1','dtheormass2','theormass2']

# dfpar = dfxls[1859:1932][col_list_print]

# col_list_print = ['state3','metha1','metha2','metha3','metha']

col_list_print = ['dvol_work_chamb[m³]','methd1','methd']

dfpar = dfxls[dfxls.num_corr==4205][dfxls.state3==1][col_list_print]

dfpar = dfxls[1857:1934][col_list_print]

# dfpar.to_excel('dfpar_4205.xls')

dfpar

### Plot and try

In [ ]:
plt.figure()
(dfxls['dvol_teor[m³]']*dfxls['state2']).plot(c='m',label='teor')
(dfxls['dvol_calc[m³]']*dfxls['state2']).plot(c='b',label='calc')
(dfxls['dvol_bal_feed_slurry[m³]']*dfxls['state2']).plot(c='cyan',label='slurry')
(dfxls['dvol_work_chamb[m³]']*dfxls['state2']).plot(c='y',label='workchamb')

plt.xlim(1855,1940)
plt.legend()

In [ ]:
dfreduced=dfxls.loc[:,[col_list[pos_num],col_list[pos_feed],col_list[pos_slurry],col_list[pos_filllev],'dvol_bal_feed_slurry[m³]','dvol_work_chamb[m³]','dvol_teor[m³]','dvol_calc[m³]','dvol_teor-calc[m³]']]

In [ ]:
dfreduced.groupby(col_list[pos_num]).agg(np.average)

In [ ]:
dfxls.loc[:,[col_list[pos_num],col_list[pos_feed],col_list[pos_slurry],col_list[pos_filllev],'dvol_bal_feed_slurry[m³]','dvol_work_chamb[m³]','dvol_teor[m³]','dvol_calc[m³]','dvol_teor-calc[m³]']]

In [ ]:
## plot "change2" that detect jump from activity to stop

dfxls[dfxls['change2']==True][[col_list[pos_stroke],col_list[pos_num],'change2','change']]

In [ ]:
## Plot "change" that detect starts of activity

dfxls[dfxls['change']==True].groupby(col_list[pos_num]).agg({col_list[pos_stroke]:min})

In [ ]:
dfxls.groupby(col_list[pos_num])[col_list[pos_stroke]].plot()

In [ ]:
plt.figure(figsize=(20,10))
dfxls.groupby('num_corr')[col_list[pos_stroke]].plot()
plt.legend()

In [ ]:
# prima metti a posto il numero di anello

In [ ]:
# used_g = df.groupby('user')['used']
# df['new'] = used_g.cumcount() - used_g.transform(lambda x: (x.values).argmax())

In [ ]:
plt.figure(figsize=(20,10))
dfxls.groupby('num_corr')['dvol_bal_feed_slurry_cum_seg[m³]'].plot()

In [ ]:
plt.figure()
(dfxls['dvol_teor[m³]']*dfxls['state2']).plot(c='m',label='teor')
(dfxls['dvol_calc[m³]']*dfxls['state2']).plot(c='b',label='calc')
(dfxls['dvol_bal_feed_slurry[m³]']*dfxls['state2']).plot(c='cyan',label='slurry')
(dfxls['dvol_work_chamb[m³]']*dfxls['state2']).plot(c='y',label='workchamb')
(dfxls['dvol_calc_MthE[m³]']).plot(c='g',label='MethE')
plt.xlim(1855,1940)
plt.legend()

In [ ]:
plt.figure()
(dfxls['dvol_teor_cum_seg[m³]']).plot(c='m',label='teor')
(dfxls['dvol_calc_cum_seg[m³]']).plot(c='b',label='calc')
(dfxls['dvol_bal_feed_slurry_cum_seg[m³]']).plot(c='cyan',label='slurry')
(dfxls['dvol_work_chamb_cum_seg[m³]']).plot(c='y',label='workchamb')
(dfxls['dvol_calc_MthE_cum_seg[m³]']).plot(c='g',label='MethE')
plt.xlim(1858,1930)
plt.legend()

In [ ]:
# 'vol_teor_cum[m³]'
# #
# 'dvol_teor[m³]'
# 'dvol_bal_feed_slurry[m³]'    
# 'dvol_work_chamb[m³]'
# #
# 'dvol_calc[m³]'
# 'dvol_teor-calc[m³]'


# dfxls['dvol_teor_cum_seg[m³]'].plot()
dfxls['dvol_bal_feed_slurry_cum_seg[m³]'].plot()
dfxls['dvol_work_chamb_cum_seg[m³]'].plot()
# dfxls['dvol_calc_cum_seg[m³]'].plot()
# dfxls['dvol_teor-calc_cum_seg[m³]'].plot()

plt.legend()

In [ ]:
plt.figure(figsize=(20,10))
(dfxls['dvol_teor_cum_seg[m³]']).plot(c='m',label='teor')
(dfxls['dvol_calc_cum_seg[m³]']).plot(c='b',label='calc')
(dfxls['dvol_bal_feed_slurry_cum_seg[m³]']).plot(c='cyan',label='slurry')
(dfxls['dvol_work_chamb_cum_seg[m³]']).plot(c='y',label='workchamb')
(dfxls['dvol_calc_MthE_cum_seg[m³]']).plot(c='g',label='MethE')
# plt.xlim(1858,1930)
plt.xlim(0,1950)
plt.legend()

In [ ]:
plt.figure(figsize=(20,8))
dfxls['dvol_teor-calc_cum_seg[m³]'].plot(c='m',label='teor')
# plt.xlim(1858,1930)
plt.xlim(0,1950)
plt.legend()

In [ ]:
plt.figure(figsize=(20,8))
dfxls[dfxls['num_corr']==4201]['dvol_teor-calc_cum_seg[m³]'].plot(c='b', label='Diff Teor-Calc')
plt.legend()

In [ ]:
col_list_print = [cl[pos_stroke],cl[pos_advspeed],'dstroke_pos[mm]','dstroke_pos_cum_seg[mm]','num_corr','change','change2','state','state1','state2','state3',cl[pos_filllev],cl[pos_feed],cl[pos_slurry],'vol_teor_cum[m³]','dvol_teor[m³]','dvol_teor_dry[m³]','bal_feed_slurry[m³/h]','dvol_bal_feed_slurry[m³]','dvol_work_chamb[m³]','dvol_calc[m³]','dvol_calc_cum_seg[m³]','dvol_calc_MthE[m³]','dvol_teor_cum_seg[m³]','dvol_calc_MthE_cum_seg[m³]','dvol_teor-calc[m³]']
# col_list_print =  [cl[pos_stroke], 'vol_teor_cum[m³]','dvol_teor[m³]']

# col_list_print = ['state3','num_corr','theormass1','dtheormass2','theormass2']

# dfpar = dfxls[1859:1932][col_list_print]

col_list_print = ['num_corr','state3','metha1','metha2','metha3','metha']


dfpar = dfxls[dfxls.num_corr==4205][dfxls.state3==1][col_list_print]

dfpar = dfxls[1857:1934][col_list_print]

# dfpar.to_excel('dfpar_4205.xls')

dfpar

In [ ]:
# 'vol_teor_cum[m³]'  DO NOT USE
# 'vol_teor_cum_dry[m³]' DO NOT USE

# 'dstroke_pos_cum_seg[mm]'
# 'dvol_teor[m³]'
# 'dvol_teor_dry[m³]'
# 'dvol_bal_feed_slurry[m³]'      feed-slurry
# 'dvol_work_chamb[m³]'           fill
# 'dvol_calc[m³]'                 dvol_bal_feed_slurry + dvol_work_chamb
# 'dvol_calc_MthE[m³]'
# 'dvol_teor-calc[m³]'
# 'dvol_teor_cum_seg[m³]'
# 'dvol_bal_feed_slurry_cum_seg[m³]'
# 'dvol_work_chamb_cum_seg[m³]'
# 'dvol_calc_cum_seg[m³]'
# 'dvol_teor-calc_cum_seg[m³]'
# 'dvol_calc_MthE_cum_seg[m³]'

In [ ]:
dfxls[dfxls['num_corr']==4205]

In [ ]:
dfxls = pd.read_hdf('ALL2.h5', key='losses')

In [ ]:
dfxls

In [ ]:
dfxls['Groutinjectionquantitycalculatedinm3[m³]']